# How to emit markdown from code cells

The code in this notebook is based on https://cran.r-project.org/web/packages/qwraps2/vignettes/summary-statistics.html It is a convenient way to create nicely formatted summary statistics.

Due to Jupyter's [security model](https://ipython.org/ipython-doc/3/notebook/security.html) we have to take a few extra steps to have results from a code cell be displayed as markdown.
* The Jupyter extension Python Markdown https://stackoverflow.com/a/43913035/4138705 can be added and that works but the results are only transient. When someone else opens the notebook, the code in the markdown cell is not run because the notebook is 'not trusted' and the reader just sees 'undefined'.
* Using `IRdisplay::display_markdown` behaves similarly and that is what we use in this notebook.
* For more detail, see https://ipython.org/ipython-doc/3/notebook/security.html

## Setup

In [1]:
lapply(c('qwraps2'),
       function(pkg) { if(! pkg %in% installed.packages()) { install.packages(pkg)} } )

Installing package into ‘/home/jupyter-user/notebooks/packages’
(as ‘lib’ is unspecified)



[[1]]
NULL

In [2]:
library(qwraps2)
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.1     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



# mtcars example

In [3]:
data(mtcars)

In [4]:
mtcars2 <-
  dplyr::mutate(mtcars,
                cyl_factor = factor(cyl,
                                    levels = c(6, 4, 8),
                                    labels = paste(c(6, 4, 8), "cylinders")),
                cyl_character = paste(cyl, "cylinders"))

In [5]:
our_summary1 <-
  list("Miles Per Gallon" =
       list("min" = ~ min(mpg),
            "max" = ~ max(mpg),
            "mean (sd)" = ~ qwraps2::mean_sd(mpg)),
       "Displacement" =
       list("min" = ~ min(disp),
            "max" = ~ max(disp),
            "mean (sd)" = ~ qwraps2::mean_sd(disp)),
       "Weight (1000 lbs)" =
       list("min" = ~ min(wt),
            "max" = ~ max(wt),
            "mean (sd)" = ~ qwraps2::mean_sd(wt)),
       "Forward Gears" =
       list("Three" = ~ qwraps2::n_perc0(gear == 3),
            "Four"  = ~ qwraps2::n_perc0(gear == 4),
            "Five"  = ~ qwraps2::n_perc0(gear == 5))
       )

table1 <- summary_table(mtcars2, our_summary1)

**If you do not see the table in the next cell**, click on the 'Not Trusted' button in the upper right hand corner of the screen.

In [6]:
IRdisplay::display_markdown(str_c(capture.output(print(summary_table(mtcars2, our_summary1), rtitle = 'TABLE 1', markup = 'markdown')),
                                           collapse = '\n'))



|TABLE 1                |mtcars2 (N = 32)    |
|:----------------------|:-------------------|
|**Miles Per Gallon**   |&nbsp;&nbsp;        |
|&nbsp;&nbsp; min       |10.4                |
|&nbsp;&nbsp; max       |33.9                |
|&nbsp;&nbsp; mean (sd) |20.09 $\pm$ 6.03    |
|**Displacement**       |&nbsp;&nbsp;        |
|&nbsp;&nbsp; min       |71.1                |
|&nbsp;&nbsp; max       |472                 |
|&nbsp;&nbsp; mean (sd) |230.72 $\pm$ 123.94 |
|**Weight (1000 lbs)**  |&nbsp;&nbsp;        |
|&nbsp;&nbsp; min       |1.513               |
|&nbsp;&nbsp; max       |5.424               |
|&nbsp;&nbsp; mean (sd) |3.22 $\pm$ 0.98     |
|**Forward Gears**      |&nbsp;&nbsp;        |
|&nbsp;&nbsp; Three     |15 (47)             |
|&nbsp;&nbsp; Four      |12 (38)             |
|&nbsp;&nbsp; Five      |5 (16)              |

# 1000 Genomes metadata example

In [7]:
sampleData <- read_csv(
    "http://storage.googleapis.com/genomics-public-data/1000-genomes/other/sample_info/sample_info.csv",
    guess_max = 3000)

dim(sampleData)


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character(),
  In_Low_Coverage_Pilot = col_double(),
  In_High_Coverage_Pilot = col_double(),
  In_Exon_Targetted_Pilot = col_double(),
  Has_Sequence_in_Phase1 = col_double(),
  In_Phase1_Integrated_Variant_Set = col_double(),
  Has_Phase1_chrY_SNPS = col_double(),
  Has_phase1_chrY_Deletions = col_double(),
  Has_phase1_chrMT_SNPs = col_double(),
  Total_LC_Sequence = col_double(),
  LC_Non_Duplicated_Aligned_Coverage = col_double(),
  Total_Exome_Sequence = col_double(),
  X_Targets_Covered_to_20x_or_greater = col_double(),
  VerifyBam_E_Omni_Free = col_double(),
  VerifyBam_E_Affy_Free = col_double(),
  VerifyBam_E_Omni_Chip = col_double(),
  VerifyBam_E_Affy_Chip = col_double(),
  VerifyBam_LC_Omni_Free = col_double(),
  VerifyBam_LC_Affy_Free = col_double(),
  VerifyBam_LC_Omni_Chip = col_double(),
  VerifyBam_LC_Affy_Chip = col_double()
  # ... with 11 more columns
)
ℹ Use `s

[1] 3500   62

In [8]:
super_populations <- sampleData %>% 
    group_by(Super_Population) %>%
    mutate(mean_Total_LC_Sequence = mean(Total_LC_Sequence, na.rm=TRUE))

In [9]:
our_summary2 <-
  list("Total Low Coverage Sequence" =
       list("min" = ~ min(mean_Total_LC_Sequence, na.rm=TRUE),
            "max" = ~ max(mean_Total_LC_Sequence, na.rm=TRUE),
            "mean (sd)" = ~ qwraps2::mean_sd(mean_Total_LC_Sequence, na_rm=TRUE)))

**If you do not see the table in the next cell**, click on the 'Not Trusted' button in the upper right hand corner of the screen.

In [10]:
IRdisplay::display_markdown(str_c(capture.output(print(summary_table(super_populations, our_summary2),
                                                      rtitle = 'TABLE 1', markup = 'markdown')),
                                  collapse = '\n'))



|TABLE 1                         |AFR (N = 1018)               |AMR (N = 535)                |EAS (N = 617)                |EUR (N = 669)                |SAS (N = 661)                |
|:-------------------------------|:----------------------------|:----------------------------|:----------------------------|:----------------------------|:----------------------------|
|**Total Low Coverage Sequence** |&nbsp;&nbsp;                 |&nbsp;&nbsp;                 |&nbsp;&nbsp;                 |&nbsp;&nbsp;                 |&nbsp;&nbsp;                 |
|&nbsp;&nbsp; min                |24252715425.9287             |23801378357.6676             |23181023724.3233             |22412145451.5541             |23485012905.6379             |
|&nbsp;&nbsp; max                |24252715425.9287             |23801378357.6676             |23181023724.3233             |22412145451.5541             |23485012905.6379             |
|&nbsp;&nbsp; mean (sd)          |24,252,715,425.93 $\pm$ 0.00 |23,801,378,357.67 $\pm$ 0.00 |23,181,023,724.32 $\pm$ 0.00 |22,412,145,451.55 $\pm$ 0.00 |23,485,012,905.64 $\pm$ 0.00 |

# Provenance

In [11]:
devtools::session_info()

─ Session info ───────────────────────────────────────────────────────────────
 setting  value                       
 version  R version 4.0.5 (2021-03-31)
 os       Ubuntu 18.04.5 LTS          
 system   x86_64, linux-gnu           
 ui       X11                         
 language (EN)                        
 collate  en_US.UTF-8                 
 ctype    en_US.UTF-8                 
 tz       Etc/UTC                     
 date     2021-05-19                  

─ Packages ───────────────────────────────────────────────────────────────────
 package     * version date       lib source        
 assertthat    0.2.1   2019-03-21 [2] CRAN (R 4.0.5)
 backports     1.2.1   2020-12-09 [2] CRAN (R 4.0.5)
 base64enc     0.1-3   2015-07-28 [2] CRAN (R 4.0.5)
 broom         0.7.6   2021-04-05 [2] CRAN (R 4.0.5)
 cachem        1.0.4   2021-02-13 [2] CRAN (R 4.0.5)
 callr         3.7.0   2021-04-20 [2] CRAN (R 4.0.5)
 cellranger    1.1.0   2016-07-27 [2] CRAN (R 4.0.5)
 cli           2.5.0   2021

Copyright 2018 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.